In [0]:
import spacy , os ,re

In [0]:
nlp = spacy.load("en_core_web_sm")

In [3]:
!wget https://github.com/DaemonLab/Cognitia-Euristica-19/raw/master/resumes.zip

--2019-04-14 13:35:18--  https://github.com/DaemonLab/Cognitia-Euristica-19/raw/master/resumes.zip
Resolving github.com (github.com)... 192.30.253.112, 192.30.253.113
Connecting to github.com (github.com)|192.30.253.112|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/DaemonLab/Cognitia-Euristica-19/master/resumes.zip [following]
--2019-04-14 13:35:18--  https://raw.githubusercontent.com/DaemonLab/Cognitia-Euristica-19/master/resumes.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 734141 (717K) [application/zip]
Saving to: ‘resumes.zip’

resumes.zip         100%[===================>] 716.93K  --.-KB/s    in 0.04s   

2019-04-14 13:35:18 (19.0 MB/s) - ‘resumes.zip’ saved [734141/734141]



In [1]:
!unzip -o resumes.zip

Archive:  resumes.zip
  inflating: resumes/sample_output.json  
  inflating: resumes/Gangadhar Vasanthapuram_Spruce InfoTech.txt  
  inflating: resumes/CV of Binnu Thomas.txt  
  inflating: resumes/Arup_Kumar_H1 B_NC.txt  
  inflating: resumes/Abbas- MS Dyna.txt  
  inflating: resumes/Amarnath.txt    
  inflating: resumes/Doc-Version/Gangadhar Vasanthapuram_Spruce InfoTech.docx  
  inflating: resumes/Doc-Version/Abbas- MS Dyna.docx  
  inflating: resumes/Doc-Version/Amarnath.doc  
  inflating: resumes/Doc-Version/CV of Binnu Thomas.doc  
  inflating: resumes/Doc-Version/Arup_Kumar_H1 B_NC.docx  


In [2]:
import os
files = os.listdir("resumes")

In [3]:
files = [f for f in files if f.endswith(".txt")]

In [4]:
files

['Amarnath.txt',
 'Arup_Kumar_H1 B_NC.txt',
 'Abbas- MS Dyna.txt',
 'Gangadhar Vasanthapuram_Spruce InfoTech.txt',
 'CV of Binnu Thomas.txt']

In [0]:
def find_email(x):
  lst = re.findall(r"[a-z0-9\.\-+_]+@[a-z0-9\.\-+_]+\.[a-z]+", x[:1000])
  return(list(set(lst)))

In [0]:
def find_contact(x):
  lst = re.findall("(\(?\d{3}\D{0,3}\d{3}\D{0,3}\d{4}).*?", x)
  temp = []
  for num in lst:
    number = num.replace("-", "")
    number = number.replace(" ", "")
    temp.append(number)
  return(list(set(temp)))

In [0]:
def find_name(x):
  doc = nlp(x[:2000])
  temp = []
  for ent in doc.ents:
    if ent.label_ == 'PERSON':
      temp.append(ent.text)
  return(temp[0])

In [0]:
def find_tech(x):
  doc = nlp(x)
  temp = []
  for ent in doc.ents:
    if ent.label_ == 'ORG':
      temp.append(ent.text)
  temp = [t for t in temp if "\t" not in t]
  temp = [t for t in temp if "\n" not in t]
  temp = [t for t in temp if "(" not in t]
  temp = [t for t in temp if "\u00a0" not in t]
  temp = [t for t in temp if "-" not in t]
  
  return(list(set(temp)))

In [0]:
import json

for file in files:
  f = open("resumes/"+str(file), "r")
  d = dict()
  doc = f.read()
  name = find_name(doc)
  contact = find_contact(doc[:1000])
  email = find_email(doc)
  tech = find_tech(doc)
  #edu = find_edu(doc)   (not implemented)
  d["name"] = name
  d["email"] = email
  d["phone"] = contact
  d["edu"] = None
  d["exp"] = tech
  
  #writing to file
  with open(str(file.replace(".txt", ""))+".json", "w") as f:
    f.write(json.dumps(d, indent=4))
  f.close()